In [ ]:
#Connect Google Colab with Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing necessery libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

In [ ]:
#Load The Data
data = pd.read_csv('/content/drive/MyDrive/Recommendation system/u.data', delimiter="\t")
data.columns = ['user_id','movie_id','rating','unix_timestamp']

In [ ]:
print(data.shape)
data.head()

(99999, 4)


,user_id,movie_id,rating,unix_timestamp
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [ ]:
#basic information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   user_id         99999 non-null  int64
 1   movie_id        99999 non-null  int64
 2   rating          99999 non-null  int64
 3   unix_timestamp  99999 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


# print out the total number of unique users and movies (print the total number of movies and the total number of users – not users and movies themselves)


In [ ]:
user_id=data['user_id'].nunique()
print(f'Number of unique Users : {user_id}')

movie_id=data['movie_id'].nunique()
print(f'Number of unique Movies : {movie_id}')

print(len(data))

Number of unique Users : 943
Number of unique Movies : 1682
99999


# create our utility matrix (user-item matrix) and print out some of its entries. The matrix will be used to compute similarity between users and items.

In [ ]:
users =data.user_id.unique().shape[0]
items =data.movie_id.unique().shape[0]
items =data['movie_id'].max()
Matrix=np.zeros((users,items))
for line in data.itertuples():
    Matrix[line[1]-1,line[2]-1] = line[3]
print(Matrix);

[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


# since our utility matrix is extremely sparse (in particular the sparsity is 
# approximately 93.7%), we need to save the memory.
# Accordingly, we have been asked to convert this giant rating matrix into a dense matrix 

In [ ]:
csr_movie_matrix=csr_matrix(Matrix)  
print(csr_movie_matrix)
#print(csr_movie_matrix[0,:3])

  (0, 0)	5.0
  (0, 1)	3.0
  (0, 2)	4.0
  (0, 3)	3.0
  (0, 4)	3.0
  (0, 5)	5.0
  (0, 6)	4.0
  (0, 7)	1.0
  (0, 8)	5.0
  (0, 9)	3.0
  (0, 10)	2.0
  (0, 11)	5.0
  (0, 12)	5.0
  (0, 13)	5.0
  (0, 14)	5.0
  (0, 15)	5.0
  (0, 16)	3.0
  (0, 17)	4.0
  (0, 18)	5.0
  (0, 19)	4.0
  (0, 20)	1.0
  (0, 21)	4.0
  (0, 22)	4.0
  (0, 23)	3.0
  (0, 24)	4.0
  :	:
  (942, 738)	4.0
  (942, 755)	2.0
  (942, 762)	4.0
  (942, 764)	3.0
  (942, 784)	2.0
  (942, 793)	3.0
  (942, 795)	3.0
  (942, 807)	4.0
  (942, 815)	4.0
  (942, 823)	4.0
  (942, 824)	3.0
  (942, 830)	2.0
  (942, 839)	4.0
  (942, 927)	5.0
  (942, 940)	1.0
  (942, 942)	5.0
  (942, 1010)	2.0
  (942, 1027)	2.0
  (942, 1043)	3.0
  (942, 1046)	2.0
  (942, 1066)	2.0
  (942, 1073)	4.0
  (942, 1187)	3.0
  (942, 1227)	3.0
  (942, 1329)	3.0


# compute similarity between movies using cosine similarity.

In [ ]:
#Create Classifier
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute',n_neighbors=3, n_jobs=-1)
model_knn.fit(csr_movie_matrix)
# n_neighbors: number of neighbors. Default is 5
# metric="cosine"

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=3)

# Now assume that we would like to generate recommendations for the user_id: 1 based on 20 movies being liked by him. Accordingly, you have been asked to: print out the 20 movies being liked by the specified user


In [ ]:
data_sort_des = data.sort_values(['user_id', 'unix_timestamp'], ascending=[True, False])
user_id_filter1 = data_sort_des[data_sort_des['user_id'] == 1].movie_id
user_id_filter1 = user_id_filter1.tolist()
user_id_filter1 = user_id_filter1[:20]
print("Items liked by user1: ",user_id_filter1)

Items liked by user1:  [74, 102, 256, 5, 171, 111, 242, 189, 32, 209, 270, 18, 6, 244, 221, 129, 20, 271, 272, 255]


# for each movie being liked by user1 (user_id = 1), recommend 2 similar movies and…That is it!

In [ ]:
distances1=[]
indices1=[]
for i in user_id_filter1:
  distances , indices = model_knn.kneighbors(csr_movie_matrix[i],n_neighbors=3)
  indices = indices.flatten()
  indices= indices[1:]
  indices1.extend(indices)
print("Items to be recommended for user1: ",indices1)

Items to be recommended for user1:  [517, 356, 230, 758, 883, 893, 473, 17, 436, 311, 615, 855, 105, 614, 103, 163, 501, 510, 642, 406, 473, 536, 578, 475, 649, 312, 688, 778, 681, 108, 275, 681, 365, 813, 685, 297, 719, 865, 373, 331]
